<a href="https://colab.research.google.com/github/yukinaga/minnano_kaggle/blob/main/section_3/01_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 特徴量エンジニアリング
データの特徴を捉え、使いやすいように加工します。  

## データの読み込み
タイタニック号の乗客データを読み込みます。  
以下のページからタイタニック号の乗客データをダウロードして、「train.csv」「test.csv」をノートブック環境にアップします。  
https://www.kaggle.com/c/titanic/data

訓練データには乗客が生き残ったどうかを表す"Survived"の列がありますが、テストデータにはありません。   


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_data = pd.read_csv("train.csv")  # 訓練データ
test_data = pd.read_csv("test.csv") # テストデータ

train_data.info()
test_data.info()

## 兄弟、配偶者の数
SibSp（兄弟/配偶者の数）の分布をグラフで確認します。

In [ ]:
sns.countplot(x="SibSp", hue="Survived", data=train_data)
plt.legend()
plt.show()

## 両親、子供の数
Parch（親/子供の数）の分布をグラフで確認します。

In [ ]:
sns.countplot(x="Parch", hue="Survived", data=train_data)
plt.legend()
plt.show()

## チケットのクラス
Pclass（チケットのクラス）の分布をグラフで確認します。

In [ ]:
sns.countplot(x="Pclass", hue="Survived", data=train_data)
plt.show()

## 性別
Sex（性別）の分布をグラフで確認します。

In [ ]:
sns.countplot(x="Sex", hue="Survived", data=train_data)
plt.show()

## 出港地
Embarked（出港地）の分布をグラフで確認します。

In [ ]:
sns.countplot(x="Embarked", hue="Survived", data=train_data)
plt.show()

## 年齢の分布
Age（年齢）の分布をグラフで確認します。  

In [ ]:
bins = 25  # 棒の数
alpha = 0.6  # 棒の透明度

# 棒グラフの描画
plt.hist(train_data.loc[train_data["Survived"]==0, "Age"].dropna(), bins=bins, alpha=alpha, label="0")  # 死亡
plt.hist(train_data.loc[train_data["Survived"]==1, "Age"].dropna(), bins=bins, alpha=alpha, label="1")  # 生存

# 軸ラベル
plt.xlabel("Age")
plt.ylabel("Count")

plt.legend()
plt.show()

## 料金の分布
Fare（料金）の分布をグラフで確認します。  

In [ ]:
bins = 25  # 棒の数
alpha = 0.6  # 棒の透明度

plt.hist(train_data.loc[train_data["Survived"] == 0, "Fare"].dropna(), bins=bins, alpha=alpha, label="0")  # 死亡
plt.hist(train_data.loc[train_data["Survived"] == 1, "Fare"].dropna(), bins=bins, alpha=alpha, label="1")  # 生存

# 軸ラベル
plt.xlabel("Fare")
plt.ylabel("Count")

plt.legend()
plt.show()

## 特徴量エンジニアリング
データを機械学習に使える形に変換し、新しい特徴量を作成します。  
まずはデータを機械学習に適した形に変換しますが、`inplace`を`True`にすることで、元のデータが変更されるようになります。

In [ ]:
data = pd.concat([train_data, test_data], sort=False)  # テストデータ、訓練データを結合

# 性別を0か1の値に変換
data["Sex"].replace(["male", "female"], [0, 1], inplace=True)

# 出港地の欠損値を埋め、0、1、2の値に変換
data["Embarked"].fillna(("S"), inplace=True)
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2})

# 欠損値を埋める
data["Fare"].fillna(data["Fare"].mean(), inplace=True)
data["Age"].fillna(data["Age"].mean(), inplace=True)

SibSpとParchを合わせて、新しい特徴量「Family」を作ります。

In [ ]:
data["Family"] = data["Parch"] + data["SibSp"]

train_data["Family"] = data["Family"][:len(train_data)]
test_data["Family"] = data["Family"][len(train_data):]

sns.countplot(x="Family", data=train_data, hue="Survived")
plt.legend()
plt.show()

不要な特徴量を削除し、機械学習モデルへの入力を作成します。

In [ ]:
# 不要な特徴量の削除
data.drop(["Name", "PassengerId", "SibSp", "Parch", "Ticket", "Cabin"],
          axis=1, inplace=True)

train_data = data[:len(train_data)]
test_data = data[len(train_data):]

t = train_data["Survived"]  # 正解
x_train = train_data.drop("Survived", axis=1)  # 訓練時の入力
x_test = test_data.drop("Survived", axis=1)  # テスト時の入力

x_train.head()